#News Article Clustering
1. Task:
Cluster thousands of news articles into related topics using a VectorDB.
2. Key Skills:
 transformer embeddings, chromadb clustering (KMeans), visualization of clusters (TSNE).
3. Challenge:
Detect and label clusters with representative keywords.


In [ ]:
!pip install langchain langchain-community chromadb sentence-transformers


In [ ]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from  langchain.text_splitter import RecursiveCharacterTextSplitter
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

In [ ]:
news = ["""AI Startup Revolutionizes Healthcare Diagnostics
A new AI-powered platform promises faster and more accurate disease detection by analyzing medical images using deep learning algorithms. Hospitals across Europe are testing its integration.""",

"""Stock Markets Rally After Central Bank Rate Cut
Global stock indices surged as central banks around the world announced coordinated interest rate cuts to stimulate economic growth amid inflationary pressures.""",


"""Researchers Discover Water Traces on Mars Crater
NASA scientists have found new evidence suggesting the presence of ancient lakes within a Martian crater, further fueling discussions about potential life on the red planet.""",


"""India Defeats Australia in Thrilling Cricket Final
India clinched the ICC Trophy after defeating Australia in a nail-biting match that went down to the final over. Fans across the country celebrated the victory.""",

"""Global Summit on Climate Change Concludes in Geneva
World leaders agreed to new emission reduction targets and pledged billions in green investments to combat global warming and promote renewable energy.""",

"""Breakthrough Vaccine Shows 95% Effectiveness Against Flu
Pharmaceutical researchers have developed a universal flu vaccine that provides long-term protection, marking a major milestone in infectious disease prevention.""",

"""New Electric Vehicle Battery Promises 1000-km Range
A Japanese automaker has unveiled a solid-state battery capable of powering electric cars for up to 1000 kilometers on a single charge, revolutionizing EV performance.""",

"""Oil Prices Drop as Supply Increases
Crude oil prices fell sharply after major producers announced plans to boost output, easing supply concerns and lowering energy costs globally.""",

"""Astronomers Detect Signal From Distant Exoplanet
Using advanced radio telescopes, astronomers picked up unusual radio emissions from a nearby star system, hinting at magnetic activity or even signs of alien technology.""",

"""Olympic Committee Announces New Sports for 2028 Games
The International Olympic Committee has approved breakdancing, surfing, and e-sports for the 2028 Games, aiming to attract younger audiences worldwide"""]


In [ ]:
label  = ["startups","finance",'astronomy',"sports","climate",'automobiles',"automobiles","finance","astronomy","sports"]

In [ ]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

docs_list = [Document(page_content=row["article"], metadata={"label": row["label"]})
             for _, row in document.iterrows()]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(docs_list)
print(f"Number of document chunks: {len(split_docs)}")



Number of document chunks: 10


In [ ]:
embedded = SentenceTransformerEmbeddings(model_name = "sentence-transformers/all-MiniLm-L6-v2")

In [ ]:
texts = [doc.page_content for doc in split_docs]
embedded_vectors = np.array(embedded.embed_documents(texts))

# 5️⃣ Store in Chroma
db = Chroma.from_documents(split_docs, embedded)

# 6️⃣ Cluster using KMeans
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(embedded_vectors)
cluster_labels = kmeans.labels_


In [ ]:
query = " Fans across the country celebrated the victory"
similar = db.similarity_search(query)
print(similar[0])

page_content='India Defeats Australia in Thrilling Cricket Final
India clinched the ICC Trophy after defeating Australia in a nail-biting match that went down to the final over. Fans across the country celebrated the victory.' metadata={'label': 'sports'}
